# 데이터 셋 생성을 위한 코드

In [ ]:
# library import 
# 필요한 라이브러리들 입니다.
# 코드 실행전에 임포트를 진행해주세요
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# data read
# data read는 드라이브 마운트가 되어있기 때문에 각자의 경로로 수정해 주셔야 합니다.
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DACON/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DACON/test.csv")

## 데이터 생성
모델에게 주어지는 데이터는 크게 4개를 생성 했습니다.  
자세한 내용은 주석 참고 부탁드려요

In [ ]:
# missing value processing
# median
# 결측치를 처리하기 위해서는 여러가지 방법이 있는데 그중 median(중간값)을 이용해야합니다.
# 평균 또는 다른 전처리 방법이 있다면 자유롭게 골라서 넣어주셔도 될 것 같아요.
train.fillna(train.median(), inplace=True)
test.fillna(train.median(), inplace=True)

# contry 열 삭제
# country 열은 따로 처리하기 힘들어서 그냥 제외를 했습니다.
train.drop(['country'], axis=1, inplace=True)
test.drop(['country'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [ ]:
# train_data_split
# train_data에는 input 데이터와 target 데이터가 나누어져 있기 때문에 쪼개주는 작업이 필요합니다.
train_input = train.iloc[:, :-1]
train_target = train.iloc[:, -1]

In [ ]:
# standard scaler
# 정규화를 한번에 하기위한 과정을 함수로 적용하였습니다.
def scaler(dataset):
  scaler = StandardScaler()
  out = scaler.fit_transform(dataset)
  return out

In [ ]:
# full data generate function
# 저희에게 주어진 전체 데이터를 이용합니다.
# 크게 수정해야할 부분은 없지만 범주형 데이터에 대해서는 원 핫 인코딩을 이용해서 이진 값으로 만들어야 할 것 같아 처리를 따로 진행햐였습니다.
def full_data(dataset):
  full_data = pd.get_dummies(dataset, columns=['gender', 'hand', 'religion', 'orientation', 'voted', 'married', 'ASD'], drop_first=True)
  full_data = scaler(full_data)
  return full_data

In [ ]:
# feature Engineering - MV data
# 마키아벨리즘 테스트를 수행하기 위해서 연관성 있는 해당 셀을 추출하여 데이터 생성(특성공학 : feature enginearing 을 진행을 합니다)
# 따로 처리를 해본 결과 별로 모델 성능 개선에는 도움이 된것 같지는 않아요.
def MV_data(dataset):
  MV_data = dataset.iloc[:, [1,2,3,4,5,7,8,9,10,11,12,13,15,16,17,19,20]]

  for i in [3,7,10,16,4,11,17,9]:
    MV_data['Q' + str(i)] = 6 - MV_data['Q' + str(i)]

  MV_data['score'] = np.sum(MV_data, axis=1)

  MV_data = scaler(MV_data)

  return MV_data

In [ ]:
# survey data - Q1~Q26
# 일반적인 설문데이터 입니다 Q1 ~ Q26 까지의 데이터를 사용하였습니다.
def survey_data(dataset):
  survey_data = dataset.iloc[:, 1:26]
  survey_data = scaler(survey_data)
  return survey_data

In [ ]:
# pca data 
# 데이터를 함수를 이용해서 생성을 한 후 차원 축소 알고리즘(pca)를 이용해서 처리를 했습니다.
# 주의해야할 부분으로는 pca데이터를 만들 때는 datase으로 바로 윗 셀에 있는 survey 데이터를 집어넣어 주셔야 합니다.
# input : survey data
def pca_data(dataset):
  poly = PolynomialFeatures(include_bias=False, degree=2)
  poly_data = poly.fit_transform(dataset)

  pca = PCA(n_components=50)
  pca_data = pca.fit_transform(poly_data)
  # pca_data = scaler(pca_data)
  return pca_data

# 모델 개발

- 모델은 각 데이터 셋 별로 3개씩 생성(모델 1개의 집단에 3개의 모델이 앙상블되어있음)
- XGBoost
- LGBM
- RandomForestClassifier

현재 데이터셋은 4개가 준비된 상태  
각각의 데이터셋을 가지고 모델 군집 4개를 생성할 예정

In [ ]:
# 모델 생성 및 예측을 자동을 수행하는 함수를 생성
# 아래 코드는 함수를 이용해서 모델 생성 및 예측을 한번에 해주는 코드입니다.
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

def model_fit_predict(dataset, targetdata, testset):

  # 여기 들어가 있는 모델은 최적화가 이루어지지 않은 데이터들 입니다.
  # 각각의 데이터와 모델들에 대해서 하이퍼파라미터최적화를 진행해 주세요!(진짜 할 수 있는 마지막 방법입니다.)

  model1 = RandomForestClassifier()
  model2 = XGBClassifier(n_estimators=2000, learning_rate=0.07, max_depth=16)
  model3 = LGBMClassifier(n_estimators=2000, max_depth=16)


  # for model in [model1, model2, model3]:
  #   model.fit(dataset,targetdata)
  
  # fit
  model1.fit(dataset,targetdata)
  model2.fit(dataset,targetdata)
  model3.fit(dataset,targetdata)
    
  # predict
  pred1 = model1.predict(testset)
  pred2 = model2.predict(testset)
  pred3 = model3.predict(testset)

  pred_sum = pred1 + pred2 + pred3

  # hard voting ensemble
  # 앙상블 모델은 sklearn의 voting 방법을 이용한 것이 아니라, 직접 코드로 구현한다.(이정도는..)
  # 1.5 이상이 된다면 1로 예측을 한다.(한개의 모델 집단에 3개의 모델이 들어 있으니까)
  predict_final = []
  for pred in pred_sum:
    if pred > 1.5:
      predict_final.append(1)
    else:
      predict_final.append(0)

  return np.array(predict_final) # 예측한 결과를 리턴한다.

## model 생성

In [ ]:
# 데이터 만들어내기
# 아까 데이터 셋을 정제하는 함수를 이용해서 진짜 데이터를 만들어 준다.
# warning 뭐시기가 뜨는데.. 그냥 무시해도 될 듯 하다. 어차피 데이터 셋은 잘 생성이 된다.
full_data_train = full_data(train_input)
full_data_test = full_data(test)

MV_data_train = MV_data(train_input)
MV_data_test = MV_data(test)

survey_data_train = survey_data(train_input)
survey_data_test = survey_data(test)

pca_data_train = pca_data(survey_data_train)
pca_data_test = pca_data(survey_data_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### 모델 생성 1개씩 찍어내자

In [ ]:
model1_pred = model_fit_predict(full_data_train, train_target, full_data_test)

In [ ]:
model2_pred = model_fit_predict(MV_data_train, train_target, MV_data_test)

In [ ]:
model3_pred = model_fit_predict(survey_data_train, train_target, survey_data_test)

In [ ]:
model4_pred = model_fit_predict(pca_data_train, train_target, pca_data_test)

### 딥러닝 모델 추가
- 사실 앙상블 모델에 딥러닝을 추가하고 싶었는데(추가를 해봤는데), 그렇게 좋은 성능을 보이지는 못했다.
- 따라서 딥러닝 모델을 생성해 내는 것 보다. 그냥 하이퍼파라미터 튜닝을 하는 방법이 더 도움이 될 것으로 판단된다.
- 일단, 아래 모델들은 각 데이터 셋에 맞추어서 모델을 생성 해 본 것들이다.
- 사실 원래 코드는 파이토치를 이용해서 작성했지만, 너무 신경써야할 부분들이 많아서 keras로 변경했다.(이정도 선형모델을 짜는데에는 케라스가 짱인듯)

In [ ]:
from tensorflow import keras

# deep_model1 with full_data
deep_model1 = keras.Sequential()
deep_model1.add(keras.layers.Dense(128, activation='leaky_relu', input_shape=(84,)))
deep_model1.add(keras.layers.Dropout(0.3))
deep_model1.add(keras.layers.Dense(64, activation='leaky_relu'))
deep_model1.add(keras.layers.Dropout(0.3))
deep_model1.add(keras.layers.Dense(32, activation='leaky_relu'))
deep_model1.add(keras.layers.Dense(16, activation='leaky_relu'))
deep_model1.add(keras.layers.Dense(8, activation='leaky_relu'))
deep_model1.add(keras.layers.Dense(1, activation='sigmoid'))

# fit
deep_model1.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics='accuracy')
deep_model1.fit(full_data_train, train_target, epochs=200)

Epoch 1/200
469/469 [==============================] - 3s 4ms/step - loss: 0.5762 - accuracy: 0.7030
Epoch 2/200
469/469 [==============================] - 2s 4ms/step - loss: 0.5465 - accuracy: 0.7227
Epoch 3/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5369 - accuracy: 0.7318
Epoch 4/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5330 - accuracy: 0.7359
Epoch 5/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5290 - accuracy: 0.7372
Epoch 6/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5281 - accuracy: 0.7427
Epoch 7/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5205 - accuracy: 0.7449
Epoch 8/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5177 - accuracy: 0.7462
Epoch 9/200
469/469 [==============================] - 1s 3ms/step - loss: 0.5140 - accuracy: 0.7481
Epoch 10/200
469/469 [==============================] - 1s 2ms/step - loss: 0.5094 - accura

In [ ]:
# deep_model2 with _MV_data
deep_model2 = keras.Sequential()
deep_model2.add(keras.layers.Dense(16, activation='leaky_relu', input_shape=(18,)))
deep_model2.add(keras.layers.Dense(8, activation='leaky_relu'))
deep_model2.add(keras.layers.Dense(4, activation='leaky_relu'))
deep_model2.add(keras.layers.Dense(2, activation='leaky_relu'))
deep_model2.add(keras.layers.Dense(1, activation='sigmoid'))

# fit
deep_model2.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics='accuracy')
deep_model2.fit(MV_data_train, train_target, epochs=100)

Epoch 1/100
469/469 [==============================] - 2s 2ms/step - loss: 0.5961 - accuracy: 0.6798
Epoch 2/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5685 - accuracy: 0.7031
Epoch 3/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5660 - accuracy: 0.7050
Epoch 4/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5651 - accuracy: 0.7074
Epoch 5/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5640 - accuracy: 0.7072
Epoch 6/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5627 - accuracy: 0.7087
Epoch 7/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5623 - accuracy: 0.7105
Epoch 8/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5617 - accuracy: 0.7089
Epoch 9/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5612 - accuracy: 0.7118
Epoch 10/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5604 - accura

In [ ]:
# deep_model3 with survey_data
deep_model3 = keras.Sequential()
deep_model3.add(keras.layers.Dense(16, activation='leaky_relu', input_shape=(25,)))
deep_model3.add(keras.layers.Dense(8, activation='leaky_relu'))
deep_model3.add(keras.layers.Dense(4, activation='leaky_relu'))
deep_model3.add(keras.layers.Dense(1, activation='sigmoid'))

# fit
deep_model3.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics='accuracy')
deep_model3.fit(survey_data_train, train_target, epochs=100)

Epoch 1/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5827 - accuracy: 0.6865
Epoch 2/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7225
Epoch 3/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5456 - accuracy: 0.7229
Epoch 4/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5430 - accuracy: 0.7239
Epoch 5/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5410 - accuracy: 0.7259
Epoch 6/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5399 - accuracy: 0.7275
Epoch 7/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7291
Epoch 8/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5372 - accuracy: 0.7298
Epoch 9/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5358 - accuracy: 0.7298
Epoch 10/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5351 - accura

In [ ]:
# deep_model4 with pca_data
deep_model4 = keras.Sequential()
deep_model4.add(keras.layers.Dense(32, activation='leaky_relu', input_shape=(50,)))
deep_model4.add(keras.layers.Dropout(0.3))
deep_model4.add(keras.layers.Dense(16, activation='leaky_relu'))
deep_model4.add(keras.layers.Dense(8, activation='leaky_relu'))
deep_model4.add(keras.layers.Dense(4, activation='leaky_relu'))
deep_model4.add(keras.layers.Dense(2, activation='leaky_relu'))
deep_model4.add(keras.layers.Dense(1, activation='sigmoid'))

# fit
deep_model4.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics='accuracy')
deep_model4.fit(pca_data_train, train_target, epochs=100)

Epoch 1/100
469/469 [==============================] - 2s 2ms/step - loss: 0.6215 - accuracy: 0.6567
Epoch 2/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5749 - accuracy: 0.6968
Epoch 3/100
469/469 [==============================] - 1s 3ms/step - loss: 0.5672 - accuracy: 0.6983
Epoch 4/100
469/469 [==============================] - 2s 4ms/step - loss: 0.5628 - accuracy: 0.7019
Epoch 5/100
469/469 [==============================] - 2s 4ms/step - loss: 0.5587 - accuracy: 0.7045
Epoch 6/100
469/469 [==============================] - 1s 3ms/step - loss: 0.5592 - accuracy: 0.7041
Epoch 7/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5566 - accuracy: 0.7076
Epoch 8/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7088
Epoch 9/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7082
Epoch 10/100
469/469 [==============================] - 1s 2ms/step - loss: 0.5514 - accura

In [ ]:
# ensemble
# 생성된 모델들을 이용해서 앙상블을 진행해본다.
def deep_result():
  final_pred = []
  preds = np.round(deep_model1.predict(full_data_test)) + np.round(deep_model2.predict(MV_data_test)) + np.round(deep_model3.predict(survey_data_test)) + np.round(deep_model4.predict(pca_data_test))

  for pred in preds:
    if pred >= 2:
      final_pred.append(1)
    else:
      final_pred.append(0)
  
  return np.array(final_pred)

In [ ]:
deep_result = deep_result()

In [ ]:
deep_result[:30]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0])

###### 찍어낸 모델 테스트
전에 0.8을 넘긴 모델은 model1, model2, model3(ML Algorithm)만을 앙상블하여 생성한 것들이다.  
따라서, 그 코드를 그대로 재현해서 올리려고 한다.

In [ ]:
# 모델이 예측한 값들을 찍어보는 코드

# 일단 model4는 당시 사용을 안해서 일단은 주석처리를 해 두었다.
print(model1_pred[:10])
print(model2_pred[:10])
print(model3_pred[:10])
# print(model4_pred[:10]) 

[0 1 1 1 0 0 1 1 0 0]
[0 1 1 1 1 0 1 0 1 0]
[0 1 1 1 1 0 1 1 0 0]


In [ ]:
# 딥러닝 예측 모델 deep_model1만 사용했다.
# 딥러닝 모델을 사용하고 싶다면 다른 것들을 추가해도 됩니다.
deep_model1_pred = np.round(deep_model1.predict(full_data_test))
deep_model1_pred = deep_model1_pred.reshape(-1)

In [ ]:
# 모델의 예측값들을 hardvoting을 위해서 더한다.
# 당시재현을 위해서 model1, model2, model3만을 이용한다.
preds = model1_pred + model2_pred + model3_pred #+ model4_pred + deep_model1_pred

In [ ]:
# 테스트로 값을 찍어본다.
preds

array([0, 3, 3, ..., 3, 0, 2])

In [ ]:
# 최종 모델의 결정을 위한 앙상블 학습
# hard voting ensemble
predict_final = []
for pred in preds:
  if pred >= 1.5:
    predict_final.append(1)
  else:
    predict_final.append(0)

result = np.array(predict_final)  # 최종 생성 결과

In [ ]:
# 제출 파일 생성
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DACON/sample_submission.csv')
submission["nerdiness"] = result

# 생성된 CSV 파일 저장하는 코드
# 코랩에서 작업시 작업 디렉터리에 생성되며 이를 로컬로 다운 받아 제출을 해 주어야 한다.
submission.to_csv("submission14.csv", index = False)